### Settings ISI project path as the absolute path, so that import functions will work for the files.

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

### Generating a list of class_names so that will be used to train and test model

In [2]:
from utils.preprocess import get_dataloaders

train_loader, test_loader, class_names = get_dataloaders()
print(class_names)  # ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']


['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from models.cnn_model import EmotionCNN
from utils.preprocess import get_dataloaders
from torchinfo import summary

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, test_loader, class_names = get_dataloaders()

# Initialize model
model = EmotionCNN(num_classes=7).to(device)
# model.summary()
# summary(model, input_size=(1, 3, 32, 32)) 

In [5]:
print(model)

EmotionCNN(
  (model): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=9216, out_features=128, bias=True)
    (8): ReLU()
    (9): Linear(in_features=128, out_features=7, bias=True)
  )
)


### Training model by settings the epoch value to 10, it can be increased to make it more efficient

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from models.cnn_model import EmotionCNN
from utils.preprocess import get_dataloaders

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, test_loader, class_names = get_dataloaders()

# Initialize model
model = EmotionCNN(num_classes=7).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train
EPOCHS = 100
for epoch in range(EPOCHS):
    model.train()
    running_loss, correct, total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {running_loss:.4f} | Accuracy: {acc:.2f}%")

torch.save(model.state_dict(), "../models/emotion_cnn.pth")

Epoch 1/100 | Loss: 1467.2895 | Accuracy: 35.02%
Epoch 2/100 | Loss: 1274.5023 | Accuracy: 45.71%
Epoch 3/100 | Loss: 1176.4418 | Accuracy: 49.92%
Epoch 4/100 | Loss: 1092.6699 | Accuracy: 53.93%
Epoch 5/100 | Loss: 1009.6049 | Accuracy: 57.67%
Epoch 6/100 | Loss: 929.4665 | Accuracy: 61.27%
Epoch 7/100 | Loss: 849.8264 | Accuracy: 65.11%
Epoch 8/100 | Loss: 769.9372 | Accuracy: 68.43%
Epoch 9/100 | Loss: 686.2452 | Accuracy: 71.99%
Epoch 10/100 | Loss: 609.3328 | Accuracy: 75.44%
Epoch 11/100 | Loss: 534.2344 | Accuracy: 78.51%
Epoch 12/100 | Loss: 461.4293 | Accuracy: 82.19%
Epoch 13/100 | Loss: 390.1549 | Accuracy: 84.70%
Epoch 14/100 | Loss: 331.2841 | Accuracy: 87.12%
Epoch 15/100 | Loss: 281.1230 | Accuracy: 89.20%
Epoch 16/100 | Loss: 232.0119 | Accuracy: 91.28%
Epoch 17/100 | Loss: 202.5445 | Accuracy: 92.35%
Epoch 18/100 | Loss: 171.4607 | Accuracy: 93.59%
Epoch 19/100 | Loss: 147.9708 | Accuracy: 94.71%
Epoch 20/100 | Loss: 136.3823 | Accuracy: 95.06%
Epoch 21/100 | Loss: 113

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from models.cnn_model import EmotionCNN
from utils.preprocess import get_dataloaders

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, test_loader, class_names = get_dataloaders()

# Initialize model
model = EmotionCNN(num_classes=7).to(device)

# ✅ Load pretrained weights before retraining
checkpoint_path = "../models/emotion_cnn.pth"
if os.path.exists(checkpoint_path):
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    print("✅ Loaded pretrained model weights.")
else:
    print("⚠️ Pretrained model not found. Training from scratch.")

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # 👈 Smaller LR for fine-tuning

# Retrain
EPOCHS = 50  # Additional epochs
for epoch in range(EPOCHS):
    model.train()
    running_loss, correct, total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {running_loss:.4f} | Accuracy: {acc:.2f}%")

# Save updated model
torch.save(model.state_dict(), "../models/emotion_cnn_retrained.pth")
print("✅ Retrained model saved.")


✅ Loaded pretrained model weights.
Epoch 1/50 | Loss: 12.8096 | Accuracy: 99.60%
Epoch 2/50 | Loss: 7.0286 | Accuracy: 99.80%
Epoch 3/50 | Loss: 6.1894 | Accuracy: 99.76%
Epoch 4/50 | Loss: 6.1534 | Accuracy: 99.74%
Epoch 5/50 | Loss: 5.8000 | Accuracy: 99.76%
Epoch 6/50 | Loss: 5.4642 | Accuracy: 99.75%
Epoch 7/50 | Loss: 5.1088 | Accuracy: 99.76%
Epoch 8/50 | Loss: 5.4087 | Accuracy: 99.76%
Epoch 9/50 | Loss: 5.4626 | Accuracy: 99.74%
Epoch 10/50 | Loss: 4.9778 | Accuracy: 99.77%
Epoch 11/50 | Loss: 5.4382 | Accuracy: 99.74%
Epoch 12/50 | Loss: 5.2491 | Accuracy: 99.74%
Epoch 13/50 | Loss: 5.2095 | Accuracy: 99.76%
Epoch 14/50 | Loss: 4.9758 | Accuracy: 99.75%
Epoch 15/50 | Loss: 5.2321 | Accuracy: 99.76%
Epoch 16/50 | Loss: 5.0200 | Accuracy: 99.75%
Epoch 17/50 | Loss: 4.9682 | Accuracy: 99.77%
Epoch 18/50 | Loss: 5.0381 | Accuracy: 99.76%
Epoch 19/50 | Loss: 5.0099 | Accuracy: 99.74%
Epoch 20/50 | Loss: 5.0716 | Accuracy: 99.75%
Epoch 21/50 | Loss: 4.8913 | Accuracy: 99.76%
Epoch 2